In [3]:
import pandas as pd

In [4]:
df = pd.read_parquet("CoinGecko_2013-04-28_2021-03-28.parquet")

In [7]:
df

01coin                                    \
              Market Cap        Volume      Open     Close   
Date                                                         
2013-04-28  1.500518e+09  0.000000e+00    135.30    141.96   
2013-04-29  1.575032e+09  0.000000e+00    141.96    135.30   
2013-04-30  1.501657e+09  0.000000e+00    135.30    117.00   
2013-05-01  1.298952e+09  0.000000e+00    117.00    103.43   
2013-05-02  1.148668e+09  0.000000e+00    103.43     91.01   
...                  ...           ...       ...       ...   
2021-03-24  1.017637e+12  6.038338e+10  54585.00  52527.00   
2021-03-25  9.827974e+11  7.302905e+10  52527.00  51417.00   
2021-03-26  9.591553e+11  6.340164e+10  51417.00  55033.00   
2021-03-27  1.027210e+12  5.544256e+10  55033.00  55832.00   
2021-03-28  1.042184e+12  4.728575e+10  55832.00       NaN   

           0-5x-long-algorand-token                                    \
                         Market Cap        Volume      Open     Close   
Date                                                                    
2013-04-28             1.500518e+09  0.000000e+00    135.30    141.96   
2013-04-29             1.575032e+09  0.000000e+00    141.96    135.30   
2013-04-30             1.501657e+09  0.000000e+00    135.30    117.00   
2013-05-01             1.298952e+09  0.000000e+00    117.00    103.43   
2013-05-02             1.148668e+09  0.000000e+00    103.43     91.01   
...                             ...           ...       ...       ...   
2021-03-24             1.017637e+12  6.038338e+10  54585.00  52527.00   
2021-03-25             9.827974e+11  7.302905e+10  52527.00  51417.00   
2021-03-26             9.591553e+11  6.340164e+10  51417.00  55033.00   
2021-03-27             1.027210e+12  5.544256e+10  55033.00  55832.00   
2021-03-28             1.042184e+12  4.728575e+10  55832.00       NaN   

           0-5x-long-altcoin-index-token                ...       zyx  \
                              Market Cap        Volume  ...      Open   
Date                                                    ...             
2013-04-28                  1.500518e+09  0.000000e+00  ...    135.30   
2013-04-29                  1.575032e+09  0.000000e+00  ...    141.96   
2013-04-30                  1.501657e+09  0.000000e+00  ...    135.30   
2013-05-01                  1.298952e+09  0.000000e+00  ...    117.00   
2013-05-02                  1.148668e+09  0.000000e+00  ...    103.43   
...                                  ...           ...  ...       ...   
2021-03-24                  1.017637e+12  6.038338e+10  ...  54585.00   
2021-03-25                  9.827974e+11  7.302905e+10  ...  52527.00   
2021-03-26                  9.591553e+11  6.340164e+10  ...  51417.00   
2021-03-27                  1.027210e+12  5.544256e+10  ...  55033.00   
2021-03-28                  1.042184e+12  4.728575e+10  ...  55832.00   

                       zzz-finance                                    \
               Close    Market Cap        Volume      Open     Close   
Date                                                                   
2013-04-28    141.96  1.500518e+09  0.000000e+00    135.30    141.96   
2013-04-29    135.30  1.575032e+09  0.000000e+00    141.96    135.30   
2013-04-30    117.00  1.501657e+09  0.000000e+00    135.30    117.00   
2013-05-01    103.43  1.298952e+09  0.000000e+00    117.00    103.43   
2013-05-02     91.01  1.148668e+09  0.000000e+00    103.43     91.01   
...              ...           ...           ...       ...       ...   
2021-03-24  52527.00  1.017637e+12  6.038338e+10  54585.00  52527.00   
2021-03-25  51417.00  9.827974e+11  7.302905e+10  52527.00  51417.00   
2021-03-26  55033.00  9.591553e+11  6.340164e+10  51417.00  55033.00   
2021-03-27  55832.00  1.027210e+12  5.544256e+10  55033.00  55832.00   
2021-03-28       NaN  1.042184e+12  4.728575e+10  55832.00       NaN   

           zzz-finance-v2                                    
               Market Cap       